In [2]:
import sys
sys.path.append('../rllab-curriculum-master/')
import curriculum.envs.arm3d.arm3d_key_env as key_env
import os.path as osp
import pickle
import numpy as np
import time
from curriculum.state.utils import StateCollection
from curriculum.envs.start_env import generate_starts
from rllab.envs.normalized_env import normalize
from curriculum.envs.goal_start_env import GoalStartExplorationEnv
from curriculum.envs.base import FixedStateGenerator

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/home/rabea/anaconda3/envs/rllab_goal_rl/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [16]:
env = key_env.Arm3dKeyEnv(ctrl_cost_coeff=[0])

start_state = lambda start_size: [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05)] if start_size == 7 else [(1.55, 0.4, -3.75, -1.15, 1.81, -2.09, 0.05, 0, 0, 0, 0, 0, 0, 0)]
    
fixed_start_generator = FixedStateGenerator(state=start_state(7))
fixed_goal_generator = FixedStateGenerator(state=[(0.0, 0.3, -0.7, 0.0, 0.3, -0.4, -0.15, 0.3, -0.55)])

inner_env = normalize(env)
start_goal_env = GoalStartExplorationEnv(env=inner_env, 
                                         start_generator=fixed_start_generator,
                                         obs2start_transform=lambda x: x[:7],
                                         goal_generator=fixed_goal_generator,
                                         obs2goal_transform=lambda x: x[-9:],
                                         terminal_eps=[0.03],
                                         distance_metric=['L2'],
                                         extend_dist_rew=[False],
                                         inner_weight=[0],
                                         goal_weight=lambda inner_weight: [1000] if inner_weight > 0 else [1],
                                         terminate_env=True
                                        )
# start_goal_env.reset()

In [14]:
print('Current obs env:', env.get_current_obs())
print()
print('Current obs innver_env:', inner_env.get_current_obs())
print()
print('Current obs start_goal_env:', start_goal_env.get_current_obs())

Current obs env: [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     1.001 -0.188 -0.25   1.001 -0.188  0.05   1.151
 -0.188 -0.1  ]

Current obs innver_env: [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     1.001 -0.188 -0.25   1.001 -0.188  0.05   1.151
 -0.188 -0.1  ]



ValueError: all the input arrays must have same number of dimensions

In [ ]:
load_dir = 'data_upload/state_collections/'
old_all_feasible_starts = pickle.load(open(osp.join(load_dir, 'all_feasible_states.pkl'), 'rb'))

In [ ]:
# print(env.kill_outside)
# print(env.kill_radius)

env.reset()
obs = env.get_current_obs()
print(obs)
print((obs == 0).sum())
print(obs[16])
print('Initialized with start state:')

init = old_all_feasible_starts.state_list[1]
# init.append(1)
print(np.array(init))
print()
print(env.reset(init_state=old_all_feasible_starts.state_list[0]))


In [ ]:
env.

In [ ]:
# in rllab/envs/mujoco/mujoco.py _get_full_obs() get_current_obs is defined
print('qpos:', env.model.data.qpos)
print()
print('qvel:', env.model.data.qvel)
print()
print('qacc:', env.model.data.qacc)
# # print('cinert shape:', env.model.data.cinert.shape)
# # print()
# # print('cvel:', env.model.data.cvel)
# # print()
# print('qfrc_actuator:', env.model.data.qfrc_actuator)
# print()
# # print('cfrc_ext:', env.model.data.cfrc_ext)
# # print()
# # print('qfrc_constraint:', env.model.data.qfrc_constraint)
# # print()
# # print('cdists:', env.model.geom_margin)
# # print()
# print('dcom:', env.dcom)

In [ ]:
for i in range(5):
    print(old_all_feasible_starts.state_list[i])
    print()

In [7]:
def find_out_feasible_states(env, log_dir=None, distance_threshold=0.1, brownian_variance=1, animate=False):
    start_time = time.time()
    no_new_states = 0
    with env.set_kill_outside():
        load_dir = 'data_upload/state_collections/'
        old_all_feasible_starts = pickle.load(open(osp.join(load_dir, 'all_feasible_states.pkl'), 'rb'))
        out_feasible_starts = StateCollection(distance_threshold=distance_threshold)
        print('number of feasible starts: ', old_all_feasible_starts.size)
        cnt = 0
        for start in old_all_feasible_starts.state_list:
            obs = env.reset(init_state=start)
#             print('Now start %i' %cnt)
            cnt += 1
            if cnt > 300:
                break
            if obs[16] > -0.5:
                # print("got one more up to ", out_feasible_starts.size)
                out_feasible_starts.append([start])
        print("number of out feasible starts:", out_feasible_starts.size)
        while no_new_states < 5:
            total_num_starts = out_feasible_starts.size
            starts = out_feasible_starts.sample(100)
            new_starts = generate_starts(env, starts=starts, horizon=1000, size=10, variance=brownian_variance,
                                         animated=animate, speedup=10)
            out_feasible_starts.append(new_starts)
            num_new_starts = out_feasible_starts.size - total_num_starts
            logger.log("number of new states: " + str(num_new_starts))
            if num_new_starts < 10:
                no_new_states += 1
#             with open(osp.join(log_dir, 'all_out_feasible_states.pkl'), 'wb') as f:
#                 cloudpickle.dump(out_feasible_starts, f, protocol=3)
    end_time = time.time()
    print('Time needed for start state generation: %.2fmin' %((end_time - start_time)/60))
    return out_feasible_starts

In [8]:
new_starts = find_out_feasible_states(start_goal_env)

number of feasible starts:  136039


ValueError: all the input arrays must have same number of dimensions